Draw a Sankey diagram!

Load the data created by the script `scripts/assemble_flows.py`:

In [ ]:
import pandas as pd
flows = pd.read_csv('data/flows.csv')

Define the Sankey diagram structure and draw:

In [ ]:
from floweaver import *

sectors = ['Construction', 'Mechanical Engineering', 'Automotive', 'Electrical',
       'Other Transport', 'Tubes', 'Metal Goods', 'Other sectors']

semis = ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
       'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes',
       'Railway track material', 'Heavy sections', 'Light sections',
       'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']

products = [('Structural', ['Structural steelwork and building and civil engineering']),
       'Mechanical engineering', 'Vehicles',
       'Electrical engineering', 'Shipbuilding',
       'Boilers, drums and other vessels','Metal goods', 
       'Other industries']

psemi = Partition.Simple('material', semis)
pprod = Partition.Simple('material', products)

pmat = Partition.Simple('material', [
    ('Flat', ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
              'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes']),
    ('Long', ['Railway track material', 'Heavy sections', 'Light sections',
              'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']),
    'Mechanical engineering', 'Electrical engineering', 'Shipbuilding',
       'Vehicles',
       ('Structural', ['Structural steelwork and building and civil engineering']),
       'Metal goods', 'Boilers, drums and other vessels',
       'Other industries'
])

nodes = {
    'prod': ProcessGroup(['uk_production'], title='UK production'),
    'semi': ProcessGroup(['uk_intermediate'], psemi, title='Semi'),
    'manuf': ProcessGroup(sectors, Partition.Simple('process', sectors), title='Manufacturing'),
    'products': ProcessGroup(['products'], pprod, title='Products'),
    'demand': ProcessGroup(['uk_demand'], title='UK demand'),
    
    'scrap': ProcessGroup(['scrap'], title='Manufacturing scrap'),
    
    'semi_export': ProcessGroup(['exports'], title='Export of semi-finished products'),
    'semi_import': ProcessGroup(['imports'], title='Import'),
    'prod_export': ProcessGroup(['product_exports'], title='Export of goods'),
    'prod_import': ProcessGroup(['product_imports'], title='Import of steel-containing goods'),
}

ordering = [
    [['semi_import'], ['prod']],
    [['semi_export'], ['semi']],
    [[], ['manuf']],
    [['prod_import'], ['products'], ['scrap']],
    [['prod_export'], ['demand']],
]

bundles = [
    Bundle('prod', 'semi'),
    Bundle('prod', 'semi_export'),
    Bundle('semi_import', 'semi'),
    Bundle('semi', 'manuf'),
    
    Bundle('manuf', 'products'),
    Bundle('manuf', 'scrap'),
    Bundle('manuf', 'prod_export'),
    Bundle('prod_import', 'demand'),
    Bundle('products', 'demand'),
    Bundle('products', 'prod_export'),
]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=pmat)
weave(sdd, flows).to_widget(width=1200, height=800, debugging=False, align_link_types=False) \
.auto_save_svg('figures/sankey_2016.svg')